In [7]:
# Install required packages.(Uncomment the below line if you have not install necessary libraries)
# pip install pandas numpy matplotlib scikit-learn joblib lightgbm xgboost seaborn scipy

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer  
from sklearn.preprocessing import OneHotEncoder      
from sklearn.model_selection import KFold   
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from statistics import mean
from sklearn.model_selection import train_test_split
import joblib 
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold

# Additional useful imports
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import learning_curve
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, median_absolute_error
import seaborn as sns
from scipy import stats
import warnings

import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# **STEP 1: DATA COLLECTION**

In [ ]:
btc_data = yf.download('BTC-USD', start='2010-01-01', end='2024-10-10', interval='1d')

In [ ]:
# Header
print("______________________Overview of the Dataset_______________________")
print("\n")

# Print out the crawled data information
print(btc_data.head(4000),"\n")
print(btc_data.info())

# **STEP 2: Prepare the incidicators**

**2.1: Add indicators to the data**

**Functions to create the indicators**

In [10]:
# Function to add indicators (e.g., moving averages, RSI)
def add_indicators(df):
    # Moving Averages (MA)
    df['MA_10'] = df['Close'].rolling(window=10).mean()  # 10-day Moving Average
    df['MA_50'] = df['Close'].rolling(window=50).mean()  # 50-day Moving Average
    df['MA_200'] = df['Close'].rolling(window=200).mean()  # 200-day Moving Average
    df['MA_20'] = df['Close'].rolling(window=20).mean()  # 20-day Moving Average (needed for Bollinger Bands)
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands (Upper and Lower Bands) - Uses MA_20
    df['BB_upper'] = df['MA_20'] + 2 * df['Close'].rolling(window=20).std()
    df['BB_lower'] = df['MA_20'] - 2 * df['Close'].rolling(window=20).std()
    
    # Exponential Moving Average (EMA)
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()  # 10-day Exponential Moving Average
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()  # 50-day Exponential Moving Average
    
    return df

Việc xử lý các giá trị NaN trong dữ liệu là một bước quan trọng trong quy trình tiền xử lý dữ liệu. Câu hỏi về việc drop (loại bỏ) các hàng với giá trị NaN hay điền các giá trị thay thế (imputation) phụ thuộc vào loại dữ liệu, bài toán cụ thể, và cách bạn muốn mô hình học từ dữ liệu.

Lý do nên drop (loại bỏ) các hàng với giá trị NaN:
Dữ liệu thiếu có thể không đáng tin cậy: Nếu giá trị thiếu chiếm một phần nhỏ và không ảnh hưởng đến mẫu dữ liệu tổng thể, việc loại bỏ có thể tránh được việc tạo ra thông tin giả mạo hoặc làm méo mó dữ liệu thật.

Chỉ báo kỹ thuật cần các giá trị liền kề: Trong trường hợp các chỉ báo như Trung bình động (MA), RSI, hay Bollinger Bands, việc điền giá trị có thể không hợp lý vì các chỉ báo này phụ thuộc vào các giá trị thực tế liền kề. Ví dụ, điền vào một giá trị NaN trong Trung bình động có thể làm sai lệch thông tin về xu hướng.

Dữ liệu thiếu không nhiều: Nếu chỉ có một số hàng chứa NaN và số lượng này không đáng kể so với toàn bộ dữ liệu, việc loại bỏ các hàng đó sẽ không ảnh hưởng lớn đến kết quả dự đoán, nhưng lại giúp tránh sự phức tạp không cần thiết trong việc điền giá trị.

Lý do nên impute (điền vào giá trị thay thế) các giá trị NaN:
Bảo toàn kích thước dữ liệu: Nếu dữ liệu chứa nhiều hàng với NaN, việc loại bỏ các hàng này có thể làm giảm kích thước bộ dữ liệu đáng kể, dẫn đến mô hình kém hiệu quả hơn hoặc không có đủ thông tin để học.

Tránh mất thông tin: Loại bỏ các hàng có thể dẫn đến việc bỏ sót những thông tin quan trọng. Nếu các giá trị NaN xảy ra rải rác, việc điền giá trị vào có thể là lựa chọn hợp lý hơn.

Phương pháp điền giá trị thông minh: Bạn có thể sử dụng các phương pháp điền giá trị tiên tiến hơn, như:

Mean/Median/Mode imputation: Điền giá trị trung bình, trung vị hoặc mode của cột.
Forward fill hoặc Backward fill: Điền giá trị NaN bằng các giá trị trước hoặc sau nó.
KNN imputation: Sử dụng KNN để dự đoán giá trị thiếu dựa trên các điểm dữ liệu lân cận.
Iterative Imputer: Điền vào giá trị thiếu bằng cách dùng các cột khác để dự đoán giá trị của nó.
Ví dụ, nếu giá trị NaN xuất hiện trong cột 'Close', bạn có thể điền giá trị bằng giá trị trung bình của các ngày liền trước và liền sau.

In [ ]:
# Add indicators to the dataset
btc_data = add_indicators(btc_data)

# Display the updated data with added indicators
print("\n______________________Dataset with Added Indicators_______________________")
print("\n")
print(btc_data.tail(10))  # Show the last 10 rows to check if indicators are added

# Drop rows with NaN values (due to moving averages and RSI calculations)
btc_data.dropna(inplace=True)

print("\n")
print(btc_data.info())

# **STEP 3: Plotting to gain some insight of the data**

In [ ]:
# Plotting price data along with Moving Averages
plt.figure(figsize=(7,4))
plt.plot(btc_data.index, btc_data['Close'], label='Close Price', color='blue', alpha=0.6)
plt.plot(btc_data.index, btc_data['MA_10'], label='MA_10', color='green', linestyle='--')
plt.plot(btc_data.index, btc_data['MA_50'], label='MA_50', color='orange', linestyle='--')
plt.plot(btc_data.index, btc_data['MA_200'], label='MA_200', color='red', linestyle='--')
plt.title('Bitcoin Price with Moving Averages (10, 50, 200 Days)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting RSI (Relative Strength Index)
plt.figure(figsize=(7,3))
plt.plot(btc_data.index, btc_data['RSI'], label='RSI', color='purple')
plt.axhline(70, linestyle='--', alpha=0.5, color='red')  # Overbought level
plt.axhline(30, linestyle='--', alpha=0.5, color='green')  # Oversold level
plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI Value')
plt.legend()
plt.grid(True)
plt.show()

# Plotting Bollinger Bands along with the Closing Price
plt.figure(figsize=(7,4))
plt.plot(btc_data.index, btc_data['Close'], label='Close Price', color='blue', alpha=0.6)
plt.plot(btc_data.index, btc_data['BB_upper'], label='Bollinger Upper Band', color='orange', linestyle='--')
plt.plot(btc_data.index, btc_data['BB_lower'], label='Bollinger Lower Band', color='orange', linestyle='--')
plt.fill_between(btc_data.index, btc_data['BB_lower'], btc_data['BB_upper'], color='orange', alpha=0.2)
plt.title('Bollinger Bands with Close Price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()


# **STEP 4: Data Preparation**

**4.1. Drop unrelated columns using MI**

4.1.1. Prepare the data for MI analysis

In [13]:
features = btc_data.drop(['Close'], axis=1)  # Drop the 'Close' column (target)
target = btc_data['Close']  # Target variable

4.1.2. Use mutual_info_regression to calculate MI between features and target

In [14]:
# Use mutual_info_regression to calculate MI between features and target
mi = mutual_info_regression(features, target, discrete_features=False)

4.1.3. Create a DataFrame to display feature importance using MI

In [15]:
mi_df = pd.DataFrame({'Feature': features.columns, 'MI_Score': mi})
mi_df = mi_df.sort_values(by='MI_Score', ascending=False)

In [ ]:
# Display the MI scores
print("\n______________________Mutual Information Scores_______________________")
print(mi_df)

4.1.3. Keep features with non-zero MI score

In [17]:
# Keep features with non-zero MI score
important_features = mi_df[mi_df['MI_Score'] > 0.05]['Feature'].tolist()

In [ ]:
print(important_features)

4.1.4. Drop unimportant features

In [19]:
btc_data_clean = btc_data[['Close'] + important_features]

In [ ]:
print("\n______________________Data After Dropping Unrelated Columns_______________________")
print(btc_data_clean.info())

**4.2. Shift label for future predictions**

In [ ]:
k = -1
btc_data['Close'] = btc_data['Close'].shift(k)

# Drop the rows with NaN values that were created by the shift
btc_data = btc_data.dropna()

# Assesment after the drop:
print("\n_____________________Data Asessment After Shifting_______________________\n")
print(btc_data.info())

**4.3. Split training the dataset**

In [ ]:
# Split the data into training and test sets
train_set, test_set = train_test_split(btc_data, test_size=0.2, random_state=42)  

# Print out some information of the split of the training set and the test set
print('\n____________ Split training and test set ____________')     
print(len(train_set), "training +", len(test_set), "test examples")
print(train_set.head(2))

**4.4 Separate labels from data, since we do not process label values (already processed)**

In [23]:
# Separate labels from data
X_train = train_set.drop(columns=['Close'])
y_train = train_set['Close']
X_test = test_set.drop(columns=['Close'])
y_test = test_set['Close']

**4.5. Define transfomer and fit the data**

Class for transformer

In [24]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    # Constructor takes a list of column names to select
    def __init__(self, feature_names):
        self.feature_names = feature_names  # Store the list of column names

    # The fit method doesn't need to do anything, it just returns self
    # to be compatible with scikit-learn's pipeline process
    def fit(self, dataframe, labels=None):
        return self

    # The transform method selects columns from the DataFrame based on the list of column names
    # and returns the values as a NumPy array
    def transform(self, dataframe):
        return dataframe[self.feature_names].values  # Select and return columns as a NumPy array


In [25]:
# Define numerical features
numerical_features = ['Open', 'High', 'Low', 'Volume', 'Adj Close', 'MA_10', 'MA_50', 'MA_200', 'RSI', 'MA_20', 'BB_upper', 
                      'BB_lower', 'EMA_10', 'EMA_50']

# Pipeline for numerical features
num_pipeline = Pipeline([
    ('selector', ColumnSelector(numerical_features)),  # Select numeric columns
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),  # Fill missing values with median
    ('std_scaler', StandardScaler(with_mean=True, with_std=True))  # Normalize to zero mean and unit variance
])

**4.6. Run the pipeline to process training data:**

In [ ]:
processed_train_set_val = num_pipeline.fit_transform(train_set)

# Fit the pipeline on training data and transform both training and test data
X_train = num_pipeline.fit_transform(X_train)
X_test = num_pipeline.transform(X_test)
X_train = pd.DataFrame(X_train, columns=numerical_features)
X_test = pd.DataFrame(X_test, columns=numerical_features)

print('\n____________ Processed feature values ____________')
print(processed_train_set_val[:3, :]) # Print out some of the first rows of the training dataset after fit_transforming
print(processed_train_set_val.shape)  # Print out the statistics of the training set
joblib.dump(num_pipeline, r'models/num_pipeline.pkl')   #  Save the pipeline 


# **STEP 5: TRAIN AND EVALUATE MODELS**

In [27]:
def r2score_and_rmse(model, train_data, labels): 
    r2score = model.score(train_data, labels)
    from sklearn.metrics import mean_squared_error
    prediction = model.predict(train_data)
    mse = mean_squared_error(labels, prediction)
    rmse = np.sqrt(mse)
    return r2score, rmse

In [28]:
def store_model(model, model_name = ""):
    # NOTE: sklearn.joblib faster than pickle of Python
    # INFO: can store only ONE object in a file
    if model_name == "": 
        model_name = type(model).__name__
    joblib.dump(model,'models/' + model_name + '_model.pkl')
    print(f"Model successfully saved as " + model_name + '_model.pkl')
    
def load_model(model_name):
    # Load objects into memory
    #del model
    model = joblib.load('models/' + model_name + '_model.pkl')
    #print(model)
    return model

**5.1. Try LightGBM**

In [ ]:
model = lgb.LGBMRegressor() #fix here
model.fit(X_train, y_train)

print('\n____________ LGBMRegressor ____________')

r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)


# Predict labels for some test instances
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

store_model(model)

**5.2. Try XGBoost model.**

In [ ]:
model = XGBRegressor() #fix here
model.fit(X_train, y_train)

print('\n____________ XGBoost_Regressor ____________')

r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)


# Predict labels for some test instances
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

store_model(model)

**5.3. Try Decision Tree.**

**NOTE:** Cây quyết định dễ bị overfitting: Nếu bạn không điều chỉnh độ sâu tối đa của cây (max_depth), mô hình có thể quá khớp với dữ liệu huấn luyện, dẫn đến điểm R² gần 1 và RMSE rất thấp trên tập huấn luyện. Tuy nhiên, khi dự đoán trên dữ liệu kiểm tra, mô hình sẽ không hoạt động tốt.

max_depth xác định độ sâu tối đa của cây quyết định. Điều này có nghĩa là cây sẽ không phát triển thêm các nhánh mới sau khi đạt đến chiều cao tối đa này.

Việc lựa chọn giá trị tối ưu cho max_depth thường được thực hiện thông qua các phương pháp tìm kiếm như Grid Search hoặc Random Search, kết hợp với k-fold cross-validation để tìm ra cấu hình tốt nhất cho mô hình.

Ngăn chặn Overfitting: Đặt giá trị max_depth thấp hơn có thể giúp ngăn ngừa việc cây quyết định quá khớp với dữ liệu huấn luyện, từ đó cải thiện khả năng tổng quát của mô hình trên dữ liệu kiểm tra.

Giảm Hiệu suất: Nếu max_depth quá thấp, mô hình có thể không đủ phức tạp để nắm bắt các mối quan hệ trong dữ liệu, dẫn đến hiệu suất kém (underfitting).

In [ ]:
model = DecisionTreeRegressor(max_depth = 10) # ĐÃ SỬA
model.fit(X_train, y_train)

print('\n____________DecisionTreeRegressor____________')

r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)


# Predict labels for some test instances
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

store_model(model)

**5.4. Try Polynomial Regression (in-lecture).**

In [32]:
# # Sử dụng PolynomialFeatures kết hợp với một mô hình hồi quy
# degree = 10  
# model = Pipeline([
#     ('poly_features', PolynomialFeatures(degree=degree)),
#     ('lin_reg', LinearRegression())
# ])

# model.fit(X_train, y_train)

# print('\n____________PolynomialRegressor____________')

# r2score, rmse = r2score_and_rmse(model, X_train, y_train)
# print('\nR2 score (on training data, best=1):', r2score)
# print("Root Mean Square Error: ", rmse)

# # Predict labels for some test instances
# print("\nPredictions: ", model.predict(X_test[:2]))
# print("Labels:      ", list(y_test[:2]))

# store_model(model)

**5.5. Try Linear Regressor (in-lecture).**

In [ ]:
model = LinearRegression() #fix here
model.fit(X_train, y_train)

print('\n____________LinearRegressor____________')

r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)

# Predict labels for some test instances
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

store_model(model)

**5.6. Try Random Forest (in-lecture).**

In [ ]:
model = RandomForestRegressor(n_estimators = 100) # ĐÃ SỬA
model.fit(X_train, y_train)

print('\n____________RandomForestRegressor____________')

r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)

# Predict labels for some test instances
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

store_model(model)

**5.7. Try K-Nearest-Neighbor**

In [ ]:
model = KNeighborsRegressor(n_neighbors=5)  # ADJUST
model.fit(X_train, y_train)

print('\n____________KNeighborsRegressor____________')

# Tính toán r2 score và rmse
r2score, rmse = r2score_and_rmse(model, X_train, y_train)
print('\nR2 score (on training data, best=1):', r2score)
print("Root Mean Square Error: ", rmse)

# Dự đoán nhãn cho một số mẫu thử nghiệm
print("\nPredictions: ", model.predict(X_test[:2]))
print("Labels:      ", list(y_test[:2]))

# Lưu mô hình đã huấn luyện
store_model(model)

## **LƯU Ý PHẢI ĐIỀU CHÌNH CÁC HỆ SỐ ĐỂ ĐƯA RA KẾT QUẢ TỐT NHẤT**

# **STEP 6: K-CROSS VALIDATION**

In [36]:
def evaluate_and_plot(model, model_name, X_train, y_train, cv):
    y_train_pred = cross_val_predict(model, X_train, y_train, cv=cv)
    nmse_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-nmse_scores)
    residuals = y_train - y_train_pred

    joblib.dump(rmse_scores, 'saved_objects/' + model_name + '_rmse.pkl')
    joblib.dump(residuals, 'saved_objects/' + model_name + '_residuals.pkl')  # Lưu residuals
    
    print(f"{model_name} RMSE: ", rmse_scores)
    print(f"Avg. RMSE: ", np.mean(rmse_scores), '\n')
    
    plt.figure(figsize=(7, 7))
    plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
    plt.title(f'Residual Distribution for {model_name} Regression')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [37]:
def load_and_print_rmse_with_plot(model_name, residuals):
    # Load RMSE scores
    rmse_scores = joblib.load('saved_objects/' + model_name + '_rmse.pkl')
    
    # Load residuals
    residuals = joblib.load('saved_objects/' + model_name + '_residuals.pkl')
    
    # Print RMSE and average RMSE
    print(f"{model_name} RMSE: ", rmse_scores)
    print(f"Avg. RMSE: ", mean(rmse_scores), '\n')
    
    # Plot residual distribution
    plt.figure(figsize=(7, 7))
    plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
    plt.title(f'Residual Distribution for {model_name} Regression')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
print('\n____________ K-fold cross validation ____________')
cv = KFold(n_splits=5,shuffle=True,random_state=37) # cv data generator

run_new_evaluation = 1
if run_new_evaluation == 1:
    models = [
        (LinearRegression(), "LinearRegression"),
        (Pipeline([('poly_features', PolynomialFeatures(degree=10)), ('lin_reg', LinearRegression())]), "PolynomialRegression"),
        (DecisionTreeRegressor(max_depth=10), "DecisionTreeRegressor"),
        (lgb.LGBMRegressor(), "LightGBM"),
        (XGBRegressor(), "XGBoostRegressor"),
        (RandomForestRegressor(n_estimators=100), "RandomForestRegressor"),
        (KNeighborsRegressor(n_neighbors=5), "KNeighbour"),
    ]
    
    for model, model_name in models:
        evaluate_and_plot(model, model_name, X_train, y_train, cv)

else:
    model_names = [
        "LinearRegression",
        "PolynomialRegression",
        "DecisionTreeRegressor",
        "RandomForestRegressor",
        "LightGBM",
        "XGBoostRegressor",
        "KNeighbour"
    ]
    
    for model_name in model_names:
        load_and_print_rmse_with_plot(model_name)
